Импорты и функции

In [2]:
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

import numpy as np
import pandas as pd
import re
import os
import shutil
import io

In [3]:
SAMPLE_SPREADSHEET_ID = '17ptHXllAQiqFhMTlWInauMA7vwwuKPY6DMfMnBGUfik'
SAMPLE_RANGE_NAME = 'A:D'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

def parse_banners(spreadsheet_id=SAMPLE_SPREADSHEET_ID, spreadsheet_range=SAMPLE_RANGE_NAME):
    creds = None

    if os.path.exists('tokens/token.json'):
        creds = Credentials.from_authorized_user_file('tokens/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'tokens/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('tokens/token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])

        return values

    except HttpError as err:
        print(err)
        return []

Импорт инфы по всем банерам и её аугментация

In [3]:
raw_data = pd.DataFrame(parse_banners())

raw_data.columns = raw_data.iloc[0]
raw_data = raw_data.drop(raw_data.index[0])

raw_data['Врачи'] = raw_data['Врачи'].replace('', method='ffill')

raw_data['CTR'] = raw_data['CTR'].str.replace('%', '').str.replace(',', '.')
raw_data['CTR'] = raw_data['CTR'].astype(float) / 100

raw_data

,Врачи,Бренд,Название файла баннера,CTR
1,Терапевты,Диклофенак-Акос,0.png,0.0107
2,Терапевты,Дексонал,1.png,0.0034
3,Педиатры,"Необутин, Максилак",2.png,0.0065
4,Фармацевты,Дексонал,3.png,0.0052
5,Фармацевты,Венарус,4.png,0.0028
6,Фармацевты,Необутин,5.png,0.0131
7,Фармацевты,Максилак,6.png,0.0039


In [6]:
df = pd.read_csv('csv/banners_info.csv')

df.drop(["CTR", "Врачи", "Бренд", "Ссылка на исходники"], axis=1).apply(lambda x: x.corr(df.CTR))

Количество символов в заголовке                      -0.548528
Количество символов в инфографике                    -0.696267
Доля инфографики на баннере                          -0.437939
Доля изображения продукта на баннере                 -0.279405
Доля изображения человека на баннере                  0.604810
Анимация                                             -0.001600
Однотонность                                         -0.019239
Указание фармакологического действия в инфографике   -0.170890
Указание преимуществ в инфорграфике                  -0.344647
Указание типа препарата в инфографике                -0.001600
dtype: float64